In [201]:
import numpy as np
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-01-17 14:35:37--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  3.30MB/s    in 0.3s    

2025-01-17 14:35:38 (3.30 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [228]:
from pathlib import Path

In [229]:
with Path("input.txt").open("r", encoding="utf-8") as f:
    text = f.read()

In [230]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab = ''.join(chars)
print(vocab_size)
print(vocab)

65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [231]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

In [232]:
# take a string and output a list of ints
encode = lambda s: [stoi[c] for c in s]
decode = lambda e: "".join([itos[i] for i in e])
# print(encode("hii there"))
# print(decode(encode("hii there")))

In [233]:
# encode(text)

In [234]:
data = torch.tensor(encode(text), dtype=torch.long)
# print(data.shape, data.dtype)
# print(data[:1000])

In [235]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [236]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
eval_iters = 200
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [237]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [251]:
@torch.no_grad() # we don't intend to do backprop so don't store all intermediate variables
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [239]:
# torch.randint(len(train_data) - block_size, (batch_size, ))
xb, yb = get_batch("train")
# print(xb)
# print(xb.shape)
# print(yb)
# print(yb.shape)

In [240]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        
        # Init at random a learnable embedding matrix
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B, T, C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for i in range(max_new_tokens):
            # print(i)
            # Get the prediction for the next token given the previous token(s)
            logits, _ = self(idx)
            # Select the predictions only
            # print('before')
            # print(logits.shape)
            # print(logits)
            logits = logits[:, -1, :] # (B, C)
            # print('after')
            # print(logits.shape)
            # print(logits)
            # Run the logits (value of embedding vector weights) through
            # softmax to get probability distribution
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution to get an index of a vector (a character)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Concatenate the idx to the current idx to create the next
            # idx vector to pass into `self` (forward pass function)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            # print()
        return idx

In [241]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)
logits, loss = m(xb, yb)
# print(logits.shape)
# print(loss)
# print(logits[:, -1])

In [242]:
emb = nn.Embedding(vocab_size, vocab_size)
emb

Embedding(65, 65)

In [243]:
# emb will lookup and return a tensor of shape (m, vocab_size) or ((B, T), vocab_size)
# of the vectors corresponding to the indices of the tensor you pass in
# val = emb(torch.tensor((5, 39, 2, 58, 29)))
# print(val.shape)

In [244]:
in_ = torch.zeros((1, 1), dtype=torch.long)
print(in_)
out = m.generate(in_, max_new_tokens=100)
# print(out)
print(decode(out[0].tolist()))

tensor([[0]])

t:toIYl;hvsVaptQU!hvGTNg;GGOyX?o.z xUgHyqjDYDjG-s,Kp&TH!:F
Xa,mWNuAMn'3p-vzuvgOY;XsHdvmvFVRlmJ'3bn$y


In [245]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [252]:
batch_size = 32
for iter_ in range(max_iters):
    if iter_ % eval_iters == 0:
        losses = estimate_loss()
        print(f"step {iter_} train_loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    # print(loss.item())
print(loss.item())

step 0 train_loss 4.6407, val loss 4.6483
step 200 train_loss 4.4183, val loss 4.4316
step 400 train_loss 4.2116, val loss 4.2182
step 600 train_loss 4.0313, val loss 4.0364
step 800 train_loss 3.8490, val loss 3.8619
step 1000 train_loss 3.6918, val loss 3.7116
step 1200 train_loss 3.5512, val loss 3.5740
step 1400 train_loss 3.4230, val loss 3.4389
step 1600 train_loss 3.3124, val loss 3.3247
step 1800 train_loss 3.1985, val loss 3.2304
step 2000 train_loss 3.1157, val loss 3.1398
step 2200 train_loss 3.0426, val loss 3.0484
step 2400 train_loss 2.9763, val loss 2.9860
step 2600 train_loss 2.9047, val loss 2.9320
step 2800 train_loss 2.8484, val loss 2.8742
2.9053354263305664


In [ ]:
in_ = torch.zeros((1, 1), dtype=torch.long, device=device)
print(in_)
out = m.generate(in_, max_new_tokens=100)
# print(out)
print(decode(out[0].tolist()))

Soft attention